In [1]:
import numpy as np
import matplotlib
import pandas as pd

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

labels = dataset.iloc[:,16:].values

vals = pd.DataFrame(dataset.iloc[:,4:15].values)

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(vals,labels,test_size=0.2,random_state=0)
X_train = np.array(X_train)


In [2]:
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor

2024-06-27 14:33:29.251568: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Initialize MLP model
def create_mlp():
    mlp_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression task
    ])

    # Compile the model
    mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), loss='mae')

    # Print model summary
    mlp_model.summary()

    # Train the model
    history = mlp_model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2)

    # Evaluate model performance
    Y_pred = mlp_model.predict(X_test)
    return mlp_model


In [4]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model


In [5]:
from tensorflow.keras.layers import concatenate

In [6]:
rf = create_rf()
mlp = create_mlp()
mlpPredict = mlp.predict(X_train)
rfPredict = rf.predict(X_train)
mlpPredict.resize(671,1)
rfPredict.resize(671,1)
trainData = concatenate([mlpPredict, rfPredict])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,393 (13.25 KB)

 Trainable params: 3,393 (13.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 34.5336 - val_loss: 23.6002
Epoch 2/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.2244 - val_loss: 21.2991
Epoch 3/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17.2192 - val_loss: 19.4449
Epoch 4/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 20.5339 - val_loss: 18.6872
Epoch 5/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.7468 - val_loss: 17.5038
Epoch 6/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.6336 - val_loss: 16.5136
Epoch 7/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.4821 - val_loss: 17.0835
Epoch 8/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.3803 - val_loss: 16.3723
Epoch 9/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.9548 - val_loss: 15.0553
Epoch 10/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.5222 - val_loss: 14.4609
Epoch 11/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 14.0443 - val_loss: 14.4210
Epoch 12/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 

In [8]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(trainData, Y_train, enable_categorical=True)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <A860C87F-F0F6-32B7-824B-DC2534A91D6C> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/o

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

Windows is not officially supported for dask/xgboost, contribution are welcomed.


TypeError: unhashable type: 'list'

In [ ]:
mlpPredictTest = mlp.predict(X_test)
rfPredictTest = rf.predict(X_test)
rfPredictTest.resize(168,1)
testData = concatenate([mlpPredictTest,rfPredictTest])
dtest_reg = xgb.DMatrix(testData, Y_test, enable_categorical=True)

In [ ]:
res = model.predict(dtest_reg)

In [ ]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(Y_test,res,squared=False)
print(rmse)